In [2]:
from datasets import load_dataset
import pandas as pd
natural_questions =load_dataset("natural_questions", split = "validation", cache_dir='/scratch/ramprasad.sa/huggingface_datasets')


In [23]:
natural_questions[0]['id']

'5225754983651766092'

In [38]:
from collections import Counter 
from tqdm import tqdm
df_dict = {
    'id': [],
    'Article': [],
    'Query': [],
    'Long_Answer': [],
    'Short_Answer': [],
    'Faithful_Query': [],
}


def get_index(answer_candidates):
    most_common_cand = Counter(answer_candidates).most_common()[0][0]
#     print(most_common_cand, answer_candidates)
    idx = answer_candidates.index(most_common_cand)
    return idx 

for datum in tqdm(natural_questions):
    docid = datum['id']
    document = datum['document']
    annotations = datum['annotations']
    question = datum['question']['text']
    html_tokens = datum['document']['tokens']['is_html']
    document_tokens = document['tokens']['token']
    answerability = 'yes'

    document = ' '.join([each for idx, each in enumerate(document['tokens']['token']) 
                    if not html_tokens[idx]])
    long_answer_candidates = [each['candidate_index'] for each in annotations['long_answer']]
    unique_long_answers = list(set(long_answer_candidates))
    short_answer_candidates = [each['start_byte'][0] for idx, each in enumerate(annotations['short_answers']) if each['start_byte']]
    short_answer_text = ''
    # print(long_answer_candidates, unique_long_answers)
    
    most_common_long_answer = get_index(long_answer_candidates)
    long_answer_info = annotations['long_answer'][most_common_long_answer]
    long_answer_text = ' '.join([document_tokens[i] for i in range(long_answer_info['start_token'], long_answer_info['end_token']) if not html_tokens[i]
                      ])
    if short_answer_candidates:
        most_common_short_answer = get_index(short_answer_candidates)
    
#         print(short_answer_candidates, Counter(short_answer_candidates).most_common())
        if annotations['short_answers'][most_common_short_answer]['start_token']:
            short_answer_start_token = annotations['short_answers'][most_common_short_answer]['start_token'][0]
            short_answer_end_token = annotations['short_answers'][most_common_short_answer]['end_token'][0]
            short_answer_text = ' '.join([document_tokens[i] for i in range(short_answer_start_token, short_answer_end_token) if not html_tokens[i]
                          ])

    if (len(unique_long_answers) == 1 and unique_long_answers[0] == -1) and (not short_answer_candidates):
        answerability = 'no'
    # if docid == '-6824623915825732866':
    #     print(annotations, unique_long_answers, (not short_answer_candidates))
    df_dict['id'] += [docid]
    df_dict['Article'] += [document]
    df_dict['Query'] += [question]
    df_dict['Long_Answer'] += [long_answer_text]
    df_dict['Short_Answer'] += [short_answer_text]
    df_dict['Faithful_Query'] += [answerability]
    # break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7830/7830 [02:20<00:00, 55.77it/s]


In [26]:
docid

'6752717162503553157'

In [34]:
df_nq[df_nq['id'] == '-6824623915825732866']

,id,Article,Query,Long_Answer,Short_Answer,Faithful_Query
4286,-6824623915825732866,Figure skating at the Olympic Games - wikipedi...,when is the figure skating part of the olympics,,,yes


In [28]:
df_nq = pd.DataFrame(df_dict)
df_nq

,id,Article,Query,Long_Answer,Short_Answer,Faithful_Query
0,5225754983651766092,Trade winds - wikipedia Trade winds Jump to : ...,what purpose did seasonal monsoon winds have o...,,,yes
1,6986236841860957647,High School Musical 2 - Wikipedia High School ...,where did they film high school musical two,,,no
2,-3290814144789249484,List of Nobel laureates in Physics - wikipedia...,who got the first nobel prize in physics,The first Nobel Prize in Physics was awarded i...,,yes
3,5745452844331879752,Alice in Wonderland ( 1951 film ) - wikipedia ...,who has the rights to alice in wonderland,,,no
4,8851020722386421469,Deadpool 2 - wikipedia Deadpool 2 Jump to : na...,when is the next deadpool movie being released,Deadpool 2 is scheduled to be released in the ...,"May 18 , 2018",yes
...,...,...,...,...,...,...
6125,1425722238049429008,Only the Brave ( 2017 film ) - wikipedia Only ...,where is the movie only the brave filmed,Principal photography on the film began on Jun...,New Mexico at different locations in and aroun...,yes
6126,-3050660247343080399,Touchdown - wikipedia Touchdown Jump to : navi...,how many points are scored for a touchdown in ...,A touchdown is worth six points . The scoring ...,six,yes
6127,-7659254827923561511,Latin Mass - wikipedia Latin Mass Jump to : na...,when did they stop saying mass in latin,A further distancing between the concepts of `...,,yes
6128,4855748823856524390,New Mexico - wikipedia New Mexico Jump to : na...,new mexico was admitted to the union as the . ...,Inhabited by Native Americans for thousands of...,47th,yes


In [21]:
df_nq.to_csv('/home/ramprasad.sa/qf_sd_summarization/datasets/postprocessed/query_focused/natural_questions.csv')



In [22]:
df_nq

,id,Article,Query,Long_Answer,Short_Answer,Faithful_Query
0,5225754983651766092,Trade winds - wikipedia Trade winds Jump to : ...,what purpose did seasonal monsoon winds have o...,,,yes
1,6986236841860957647,High School Musical 2 - Wikipedia High School ...,where did they film high school musical two,,,no
2,-3290814144789249484,List of Nobel laureates in Physics - wikipedia...,who got the first nobel prize in physics,The first Nobel Prize in Physics was awarded i...,,yes
3,5745452844331879752,Alice in Wonderland ( 1951 film ) - wikipedia ...,who has the rights to alice in wonderland,,,no
4,8851020722386421469,Deadpool 2 - wikipedia Deadpool 2 Jump to : na...,when is the next deadpool movie being released,Deadpool 2 is scheduled to be released in the ...,"May 18 , 2018",yes
...,...,...,...,...,...,...
7825,6063856819417468862,Timeline of the Ottoman Empire - wikipedia Tim...,when did the ottoman empire start and end,,,no
7826,-3290922489528991816,Siliguri Junction railway station - Wikipedia ...,which of the following stations has all the th...,,,no
7827,-430859680692445019,Home computer - wikipedia Home computer Jump t...,when did computer become widespread in homes a...,,,yes
7828,5474302889028409102,Stay ( Shakespears Sister song ) - wikipedia S...,who sings the 80 song stay with me,,,no
